# PoC for walking bassline generator

## 1. Parse chords progressions
- Input: string with chord progression in American cipher
- Output: Chords objects (detecting different intervals that will be used to create the bass lines)
          Chords components (1st, 3rd, 5th, 7th)

Will use [pychord](https://pypi.org/project/pychord/).

In [20]:
from pychord import Chord, ChordProgression

In [21]:
chord_str = 'Cm'

chord = Chord(chord_str)

In [22]:
chord.components()

['C', 'Eb', 'G']

In [23]:
print(chord.info())

Cm
root=C
quality=m
appended=[]
on=


In [24]:
chord_progression_str = 'C7 Am7 Fmaj7 G7'

In [25]:
chord_progression = ChordProgression(chord_progression_str.split())

In [26]:
chord_progression.chords

[<Chord: C7>, <Chord: Am7>, <Chord: Fmaj7>, <Chord: G7>]

In [27]:
for ch in chord_progression.chords:
    display(ch.components())

['C', 'E', 'G', 'Bb']

['A', 'C', 'E', 'G']

['F', 'A', 'C', 'E']

['G', 'B', 'D', 'F']

## 2. Bassline figures
- Input: None
- Output: Different options for bassline figures (e.g. combination of eighth and sixteenth note)

In [1]:
import pretty_midi

In [10]:
bass_line = pretty_midi.PrettyMIDI(initial_tempo=120.0)

In [11]:
# Create an Instrument instance for a bass instrument
bass_program = pretty_midi.instrument_name_to_program('Acoustic Bass') # 'Electric Bass'
bass = pretty_midi.Instrument(program=bass_program)

In [12]:
ts = pretty_midi.TimeSignature(2, 4, 0.0)
print('Time Signature >', ts)

Time Signature > 2/4 at 0.00 seconds


In [13]:
bass_line.get_downbeats(0.0)

array([], dtype=float64)

👆 will not use `pretty_midi`

---

👇 will try `partitura` instead

In [63]:
import partitura.score as score
import partitura

In [64]:
qd = 8
bassline = score.Part('BL0', 'Bassline', quarter_duration=qd)
bassline.add(score.TimeSignature(2, 4), start=0)

In [65]:
print(bassline.pretty())

Part id="BL0" name="Bassline"
 │
 └─ TimePoint t=0 quarter=8
     │
     └─ starting objects
         │
         └─ 0-- TimeSignature 2/4


In [66]:
n = 0
for ch in chord_progression.chords:
    for note in ch.components():
        note_id = 'n{}'.format(n)
        alter = None
        if len(note) > 1:
            if note[1] == 'b':
                alter = -1
            elif note[1] == '#':
                alter = 1
        bassline.add(score.Note(id=note_id, step=note[0], alter=alter, 
                                octave=3, voice=None), 
                     start=n*qd, end=(n+1)*qd)
        n += 1

score.add_measures(bassline)

In [67]:
print(bassline.pretty())

Part id="BL0" name="Bassline"
 │
 ├─ TimePoint t=0 quarter=8
 │   │
 │   └─ starting objects
 │       │
 │       ├─ 0--16 Measure number=1
 │       ├─ 0--8 Note id=n0 voice=None staff=None type=quarter pitch=C3
 │       └─ 0-- TimeSignature 2/4
 │
 ├─ TimePoint t=8 quarter=8
 │   │
 │   ├─ ending objects
 │   │   │
 │   │   └─ 0--8 Note id=n0 voice=None staff=None type=quarter pitch=C3
 │   │
 │   └─ starting objects
 │       │
 │       └─ 8--16 Note id=n1 voice=None staff=None type=quarter pitch=E3
 │
 ├─ TimePoint t=16 quarter=8
 │   │
 │   ├─ ending objects
 │   │   │
 │   │   ├─ 0--16 Measure number=1
 │   │   └─ 8--16 Note id=n1 voice=None staff=None type=quarter pitch=E3
 │   │
 │   └─ starting objects
 │       │
 │       ├─ 16--32 Measure number=2
 │       └─ 16--24 Note id=n2 voice=None staff=None type=quarter pitch=G3
 │
 ├─ TimePoint t=24 quarter=8
 │   │
 │   ├─ ending objects
 │   │   │
 │   │   └─ 16--24 Note id=n2 voice=None staff=None type=quarter pitch=G3
 │   │
 │   └─

In [69]:
# partitura.render(bassline)
partitura.save_musicxml(bassline, 'bassline.musicxml')

In [71]:
partitura.save_score_midi(bassline, 'bassline.mid')

## 3. Visualizing Tabs
- Input: Music score in `musicxml` format, or MIDI file
- Output: Bass/Guitar tab in text format.

In [70]:
from tayuya import MIDIParser

In [74]:
mid = MIDIParser('bassline.mid', track=0)
mid.get_tracks()

{0: ''}

In [77]:
#mid.render_tabs()